In [3]:
######################################################## Import libraries ######################################################
import ee
import sys
import os
#import tensorflow as tf
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
import math
import datetime
import time
import pandas as pd
from dateutil.relativedelta import *
from gee_prep import filter_collection, rescale_rename_landsat, export_to_drive, get_gap_filled_image, preprocess_modis, get_output_samples
from gee_prep import reproject_collection, clip_collection
from landsat8_prep import update_qamask_8, rescale_rename_landsat8
ee.Initialize()
######################################################### Define variables #####################################################
landsat8_collection = 'LANDSAT/LC08/C02/T1_L2'
modis_collection = 'MODIS/061/MYD11A1'
city_collection = 'projects/ee-yiwenz/assets/GUB_Global_2018_gt_25_sorted'
#city_400_file='/glade/u/home/yiwenz/TransferLearning/Forcing_scripts/largest_400_cities_new.csv'
cities = ee.FeatureCollection(city_collection)
cities=cities.sort("urbanArea",False)
predictors = ['Blue','Green','Red','NIR','SWIR1','SWIR2','ST']
landsat_res = 30 #m
radius = 16 #(33-1)/2 pixels
#cities_400=pd.read_csv(city_400_file)[['City_Name','NAME10']]

In [4]:
cities.size().getInfo()

3982

In [2]:
ee.Authenticate()

Enter verification code:  4/1ASVgi3Ix7dfguJRixcRaB6L1ouTeLuDoDmy5s_D31jf4SylUHVBcD0Dx8l4



Successfully saved authorization token.


# Download Landsat and MODIS

In [15]:
############################################ Process Landsat8 1 month #####################################################
city_idx_start,city_idx_end=3900,3982

# start_landsat = datetime.date(2013, 4, 1)
# end_landsat = datetime.date(2019, 1, 1)
start_landsat = datetime.date(2019, 1, 1)
end_landsat = datetime.date(2024, 1, 1)
months = range(1,13)

predictors = ['Blue','Green','Red','NIR','SWIR1','SWIR2']
cities_list=cities.toList(cities.size())

for city_idx in range(city_idx_start,city_idx_end):
    city_name = ee.Feature(cities_list.get(city_idx)).get("ORIG_FID").getInfo()
    target_city= cities.filter(ee.Filter.eq('ORIG_FID',city_name))
    bound=target_city.geometry().buffer(2000)
    print(f'processing {city_name}')
    #create asset folder
    try:
        ee.data.createAsset({'type': 'Folder'}, f'users/yiwenz9/{city_name}_Landsat8')
    except:
        print('Folder already exists.')

    modis = filter_collection(modis_collection, start_date='2013-04-01', end_date='2013-04-02',geometry=target_city)
    landsat = filter_collection(landsat8_collection, start_date=str(start_landsat), end_date=str(end_landsat),geometry=target_city)\
            .map(lambda image: reproject_collection(image,modis.first().projection(),scale=30))\
            .map(update_qamask_8)\
            .map(rescale_rename_landsat8)\
            .sort('CLOUD_COVER')\
            .select(predictors)
    for month in months:
        print(month)
        landsat_download=landsat\
            .filter(ee.Filter.calendarRange(month,month,'month'))
        img_export=landsat_download\
                .median()\
                .reproject(landsat.first().projection())\
                .unmask(-9999)
        img_name=f'users/yiwenz9/{city_name}_Landsat8/{city_name}_{str(month)}_2019-2024'
        task = ee.batch.Export.image.toAsset(
                image=img_export,
                description=f'landsat_{city_name}_{month}_asset',
                assetId=img_name,
                region=bound)
        task.start()

processing 48073
1
2
3
4
5
6
7
8
9
10
11
12
processing 2413
1
2
3
4
5
6
7
8
9
10
11
12
processing 18799
1
2
3
4
5
6
7
8
9
10
11
12
processing 50192
1
2
3
4
5
6
7
8
9
10
11
12
processing 30819
1
2
3
4
5
6
7
8
9
10
11
12
processing 8526
1
2
3
4
5
6
7
8
9
10
11
12
processing 23569
1
2
3
4
5
6
7
8
9
10
11
12
processing 56673
1
2
3
4
5
6
7
8
9
10
11
12
processing 1889
1
2
3
4
5
6
7
8
9
10
11
12
processing 61667
1
2
3
4
5
6
7
8
9
10
11
12
processing 37593
1
2
3
4
5
6
7
8
9
10
11
12
processing 33119
1
2
3
4
5
6
7
8
9
10
11
12
processing 15965
1
2
3
4
5
6
7
8
9
10
11
12
processing 44885
1
2
3
4
5
6
7
8
9
10
11
12
processing 14499
1
2
3
4
5
6
7
8
9
10
11
12
processing 17789
1
2
3
4
5
6
7
8
9
10
11
12
processing 22213
1
2
3
4
5
6
7
8
9
10
11
12
processing 23019
1
2
3
4
5
6
7
8
9
10
11
12
processing 63836
1
2
3
4
5
6
7
8
9
10
11
12
processing 2
1
2
3
4
5
6
7
8
9
10
11
12
processing 64421
1
2
3
4
5
6
7
8
9
10
11
12
processing 44217
1
2
3
4
5
6
7
8
9
10
11
12
processing 62613
1
2
3
4
5
6
7
8
9
10
1

In [8]:
####################################### Modis to TFRecord #####################################################
#task_recent = ee.data.getTaskStatus("task id here")
city_idx_start,city_idx_end=2344,2400
LST_band=['LST_Day_1km','LST_Night_1km']
QC_band=['QC_Day','QC_Night']
# time_pair = [str(datetime.date(2013, 4, 1)),str(datetime.date(2019, 1, 1))]
time_pair = [str(datetime.date(2019, 1, 1)),str(datetime.date(2024, 1, 1))]
months = range(1,2)
# months = range(2,13)

def add_prop(feature,date):
    feature_id=feature.id()
    return feature.set('date',date).set('id',feature_id)

cities_list=cities.toList(cities.size())

for city_idx in range(city_idx_start,city_idx_end):
    city_name = ee.Feature(cities_list.get(city_idx)).get("ORIG_FID").getInfo()
    target_city= cities.filter(ee.Filter.eq('ORIG_FID',city_name))
    drive_folder = f'Data_TFRecord_Daily_{str(city_name)}_Landsat8_Day_Night_926_Composite_2019-2024'

    print(f'processing {city_name}')

    start_modis,end_modis = time_pair    
    for month in months:
        print(month)    
        export_list=ee.List([])
        modis=preprocess_modis(modis_collection, start_date=start_modis, end_date=end_modis, geometry=target_city,
                               LST_band=LST_band, QC_band=QC_band)\
              .filter(ee.Filter.calendarRange(month,month,'month'))
        modis_list = modis.toList(modis.size())
        for i in range(modis.size().getInfo()):
            modis_download = ee.Image(modis_list.get(i))
            vector=modis_download.sample(
              region= target_city.geometry(),
              dropNulls=False,
              geometries=True
            )
            fname=ee.Date(modis_download.get('system:time_start')).format("YYMMdd")
            vector = vector.map(lambda feature: add_prop(feature,fname))
            collection_list = vector.toList(vector.size())
            export_list = export_list.add(collection_list) 
        collection_export=ee.FeatureCollection(export_list.flatten())
        description = f'modis_{city_name}_{month}'
        file_name = f'modis_{month}'
        task = ee.batch.Export.table.toDrive(
            collection=collection_export,
            description=description,
            folder=drive_folder,
            fileNamePrefix=file_name,
            fileFormat='TFRecord')
        task.start()

processing 32465
1
processing 20010
1
processing 35157
1
processing 50882
1
processing 30769
1
processing 36428
1
processing 16850
1
processing 29286
1
processing 13705
1
processing 8257
1
processing 26524
1
processing 44985
1
processing 17799
1
processing 41249
1
processing 52315
1
processing 7738
1
processing 52400
1
processing 3675
1
processing 52987
1
processing 10509
1
processing 15349
1
processing 12582
1
processing 32564
1
processing 1412
1
processing 6398
1
processing 58355
1
processing 9730
1
processing 22125
1
processing 10359
1
processing 42057
1
processing 56472
1
processing 39001
1
processing 44790
1
processing 1420
1
processing 58325
1
processing 7316
1
processing 48869
1
processing 29416
1
processing 45409
1
processing 56734
1
processing 55085
1
processing 38584
1
processing 6371
1
processing 62266
1
processing 7439
1
processing 49940
1
processing 23043
1
processing 48033
1
processing 64356
1
processing 15649
1
processing 18673
1
processing 14514
1
processing 43562
1
pro

In [7]:
####################################### Landsat8 to TFRecord #####################################################
def add_prop(feature):
    feature_id=feature.id()
    return feature.set('id',feature_id)

city_idx_start,city_idx_end=3900,3982 #165
months = range(1,13)

cities_list=cities.toList(cities.size())

for city_idx in range(city_idx_start,city_idx_end):
    city_name = ee.Feature(cities_list.get(city_idx)).get("ORIG_FID").getInfo()
    target_city= cities.filter(ee.Filter.eq('ORIG_FID',city_name))
    drive_folder = f'Data_TFRecord_Daily_{str(city_name)}_Landsat8_Day_Night_926_Composite_2019-2024'
    print(drive_folder)

    # modis=preprocess_modis(modis_collection, start_date='2013-04-01', end_date='2013-04-02', geometry=target_city)
    modis=preprocess_modis(modis_collection, start_date='2019-01-01', end_date='2019-01-02', geometry=target_city)
    vector=modis.first().sample(
      region= target_city.geometry(),
      dropNulls=False,
      geometries=True
    )
    vector = vector.map(lambda feature: add_prop(feature))
    
    print(f'processing {city_name}')

    for month in months:
        print(month)
        try:
            file_name=f'users/yiwenz9/{city_name}_Landsat8/{city_name}_{str(month)}_2019-2024'
            img_export=ee.Image(file_name)
            collection = get_output_samples(img = img_export, region = vector, radius = radius, units = 'pixels', scale = landsat_res)
            description = f'landsat_{city_name}_{month}_drive'
            file_name = f'landsat_{month}'
            task = ee.batch.Export.table.toDrive(
                collection=collection,
                description=description,
                folder=drive_folder,
                fileNamePrefix=file_name,
                fileFormat='TFRecord')
            task.start()
        except:
            print('File does not exist.')

Data_TFRecord_Daily_48073_Landsat8_Day_Night_926_Composite_2019-2024
processing 48073
1
2
3
4
5
6
7
8
9
10
11
12
File does not exist.
Data_TFRecord_Daily_2413_Landsat8_Day_Night_926_Composite_2019-2024
processing 2413
1
2
3
4
5
6
7
8
9
10
11
12
Data_TFRecord_Daily_18799_Landsat8_Day_Night_926_Composite_2019-2024
processing 18799
1
2
3
4
5
6
7
8
9
10
11
12
Data_TFRecord_Daily_50192_Landsat8_Day_Night_926_Composite_2019-2024
processing 50192
1
2
3
4
5
6
7
8
9
10
11
12
Data_TFRecord_Daily_30819_Landsat8_Day_Night_926_Composite_2019-2024
processing 30819
1
2
3
4
5
6
7
8
9
10
11
12
Data_TFRecord_Daily_8526_Landsat8_Day_Night_926_Composite_2019-2024
processing 8526
1
2
3
4
5
6
7
8
9
10
11
12
Data_TFRecord_Daily_23569_Landsat8_Day_Night_926_Composite_2019-2024
processing 23569
1
2
3
4
5
6
7
8
9
10
11
12
Data_TFRecord_Daily_56673_Landsat8_Day_Night_926_Composite_2019-2024
processing 56673
1
2
3
4
5
6
7
8
9
10
11
12
Data_TFRecord_Daily_1889_Landsat8_Day_Night_926_Composite_2019-2024
processing 

In [ ]:
city_collection = 'projects/ee-yiwenz/assets/GUB_Global_2018_gt_25_sorted'
#city_400_file='/glade/u/home/yiwenz/TransferLearning/Forcing_scripts/largest_400_cities_new.csv'
cities = ee.FeatureCollection(city_collection)
cities_sorted = cities.sort("urbanArea",False)
name_list=cities_sorted.aggregate_array('ORIG_FID').getInfo()
area_list=cities_sorted.aggregate_array('urbanArea').getInfo()
df=pd.DataFrame(name_list).rename(columns={0:'City_Name'})
df["Area"]=area_list
df.to_csv("/glade/u/home/yiwenz/TransferLearning/City_NameList/GUB_Global_2018_gt_25_sorted_new.csv",index=False)
df

In [ ]:
#Export Modis grids
city_idx_start,city_idx_end=[0,400]

def add_prop(feature):
    coords=feature.geometry().coordinates()
    return feature.set('lon',coords.get(0)).set('lat',coords.get(1))

cities_list=cities.toList(cities.size())

for city_idx in range(city_idx_start,city_idx_end):
    city_name = ee.Feature(cities_list.get(city_idx)).get("ORIG_FID").getInfo()
    target_city= cities.filter(ee.Filter.eq('ORIG_FID',city_name))
    print(city_name)
    modis = ee.ImageCollection(modis_collection).filterBounds(target_city).filterDate('2002-09-01').first().select('LST_Day_1km').clip(target_city)
    vector_out=modis.sample(
                              region= target_city.geometry(),
                              geometries=True,
                              dropNulls=False
                            )
    vector_out=vector_out.map(lambda feature: add_prop(feature))
    vector_data = vector_out.map(lambda f: ee.Feature(None, f.toDictionary()))
    task = ee.batch.Export.table.toDrive(
        collection=vector_data,
        folder=f"modis_grid",
        description='modis_grid_'+city_name,
        fileNamePrefix='modis_grid_'+city_name)
    task.start()    

In [ ]:
##Store modis grid in correct format
city_400_file='/glade/u/home/yiwenz/TransferLearning/City_NameList/china_400.csv'
city_idx_start,city_idx_end=[0,400]
cities_400=pd.read_csv(city_400_file)
for city_idx in range(city_idx_start,city_idx_end):
    city_name=str(int(cities_400.loc[city_idx].City_Name))
    print(city_name)
    PATH_TO_OLD_MODIS_LAT_LON=f'/glade/work/yiwenz/modis_lat_lon_China_926/modis_grid_{city_name}.csv'
    PATH_TO_STORE_MODIS_LAT_LON=f'/glade/work/yiwenz/modis_lat_lon_China_926/modis_grid_{city_name}.csv'
    df_data=pd.read_csv(PATH_TO_OLD_MODIS_LAT_LON)[['lat','lon']]
    df_data.to_csv(PATH_TO_STORE_MODIS_LAT_LON,index=False)

# Delete dataset

In [ ]:
### Delete Landsat from GEE ###
import ee
import time
import pandas as pd
ee.Initialize()
city_idx_start,city_idx_end=0,384
city_400_file='/glade/u/home/yiwenz/TransferLearning/Forcing_scripts/largest_400_cities_new.csv'
cities_400=pd.read_csv(city_400_file)[['City_Name','NAME10']]

for city_idx in range(city_idx_start,city_idx_end):
    city_list=cities_400.loc[city_idx].to_list()
    city_folder, city_name = city_list
    print(city_folder)
    parent_folder='projects/ee-yiwenz/assets/'+city_folder+'_Landsat8' #projects/ee-yiwenz/assets/ #users/yiwenz9/
    try:
        parent_asset=ee.data.getAsset(parent_folder)
        parent_id = parent_asset['name']
        parent_type = parent_asset['type']
        asset_list = []
        child_assets = ee.data.listAssets({'parent': parent_id})['assets']
        for asset in child_assets:
            ee.data.deleteAsset(asset['id'])
        ee.data.deleteAsset(parent_folder)
    except:
        print(f'Asset does not exist for {city_name}')

In [16]:
### Delete Landsat from GEE ###
import ee
import time
import pandas as pd
ee.Initialize()
city_idx_start,city_idx_end=1800,2000

cities_list=cities.toList(cities.size())
for city_idx in range(city_idx_start,city_idx_end):
    target_city= ee.Feature(cities_list.get(city_idx))
    city_name = str(target_city.get("ORIG_FID").getInfo())
    print(city_name)
    parent_folder=f'users/yiwenz9/{city_name}_Landsat8'
    try:
        parent_asset=ee.data.getAsset(parent_folder)
        parent_id = parent_asset['name']
        parent_type = parent_asset['type']
        asset_list = []
        child_assets = ee.data.listAssets({'parent': parent_id})['assets']
        for asset in child_assets:
            ee.data.deleteAsset(asset['id'])
        ee.data.deleteAsset(parent_folder)
    except:
        print(f'Asset does not exist for {city_name}')

6776
59990
29363
42790
34549
35156
33232
38808
7015
52290
8105
36226
35203
20683
54092
18163
36736
11519
9682
46753
36705
59543
60845
42139
119
59725
51039
55683
53775
24330
22323
59798
41383
19235
9429
12792
47176
64007
41853
16205
13145
45779
41903
47372
1811
51425
29429
17691
8780
59727
57660
14583
49018
1831
594
15687
23706
369
23370
32556
10186
63113
61879
7799
4936
3466
7365
27475
37406
57458
36524
11836
32903
50535
38076
20539
32634
48184
6283
48311
54710
18057
56975
7118
40189
50574
50761
28338
36272
5439
22723
53927
55103
55032
61425
16368
21463
19612
14261
39540
50791
15090
47051
9694
13436
27341
8408
20017
40685
7280
10121
7693
44014
32839
2360
41215
48745
36519
11086
4224
13169
46048
33073
12938
29393
31185
14180
37449
54795
18593
6755
65184
60089
57777
43194
4952
54102
26220
38064
43155
25155
15508
27997
11433
10015
51984
42768
64749
35446
19622
33075
2096
890
27775
35970
3494
38996
32123
51809
23599
7652
54220
11924
51285
54689
36260
5404
45672
32358
40301
248
53233
34520

# Other stuff

In [ ]:
city_idx_start,city_idx_end=[8,9]

city_400_file='/glade/u/home/yiwenz/TransferLearning/Forcing_scripts/largest_400_cities_new.csv'
cities_400=pd.read_csv(city_400_file)[['City_Name','NAME10','Time_zone']]
outDir_modis_grid='/glade/work/yiwenz/modis_xy_US_cities'

for city_idx in range(city_idx_start,city_idx_end):
    city_list=cities_400.loc[city_idx].to_list()
    city_name, city_num, city_hour=city_list
    target_city=cities.filter(ee.Filter.eq('NAME10',city_num))
    PATH_TO_STORE_MODIS_LAT_LON=os.path.join(outDir_modis_grid,'modis_grid_'+city_name+'.csv')
    print(city_name)
#    if not os.path.exists(PATH_TO_STORE_MODIS_LAT_LON):
    modis = filter_collection(modis_collection, start_date='2002-09-01', end_date='2002-09-02',geometry=target_city.geometry())
    projection=modis.first().projection()
    modis_reproj=modis\
                .map(lambda image: clip_collection(image, target_city))\
                .filterBounds(target_city)\
                .select('LST_Day_1km')
    vector_out=modis_reproj.first().pixelCoordinates(projection).sample(
                                                              region= target_city.geometry(),
                                                              geometries=True,
                                                              dropNulls=False
                                                            )
    vector_data = vector_out.map(lambda f: ee.Feature(None, f.toDictionary()))
    vector_data = [x["properties"] for x in vector_data.getInfo()["features"]]
    vector_df = pd.DataFrame(vector_data)*1000
#    vector_df.to_csv(PATH_TO_STORE_MODIS_LAT_LON)

In [ ]:
## Examine TFRecord
from tfrecords2numpy_modis_day_night import TFRecordsParser as TFRecordsParser_Modis
file='modis_1.tfrecord'
records_modis = TFRecordsParser_Modis(file).tfrecrods2numpy()

In [ ]:
records_modis[1829]

In [ ]:
from tfrecords2numpy_landsat_no_idx import TFRecordsParser
file='landsat_1.tfrecord'
records_landsat = TFRecordsParser(file).tfrecrods2numpy()

In [ ]:
len(records_landsat)